In [ ]:
# posterior plots
import sys
sys.path.append('../../')
from fig1_calibration import create_sim as cs
import pandas as pd
import sciris as sc
from matplotlib import rc
from scipy.stats import qmc
import matplotlib.pyplot as plt
#%config InlineBackend.figure_formats = ['svg']
rc('font', **{'family': 'serif', 'serif': ['Times']})
rc('text', usetex=False)
rc('grid', linestyle="--", color='black')


bounds = cs.define_pars(which='bounds',use_safegraph=True)
bounds['tn'][0] = 5.0
start_day = pd.to_datetime('2020-01-27')

In [ ]:
sims = pd.read_csv('../data/sims_combined-posterior.csv')
df = sims.copy()
df

In [ ]:
import numpy as np
qs = [0.025,0.975,0.5,0.25,0.75] 
cols = ['cum_diagnoses','cum_deaths','diagnoses','death','infectious']
df_plot = (
    df.groupby('t')
    [cols].quantile(qs)
.reset_index()
.pivot(index='t',
columns=['level_1'],
values=cols
))
df_plot

In [ ]:
# plot posterior
import matplotlib.ticker as ticker
import datetime as dt
import seaborn as sns
fig, ax = plt.subplot_mosaic(mosaic=[['a']*4,
                                     ['b']*4,
                                     ['c']*4,
                                     ['d','e','f','g']],figsize=(11.5,8.5))
colors = ['blue','red','green']
n_days = sims[sims['rand_seed']==sims['rand_seed'].min()].shape[0]
def format_ax(ax):
    ''' Format the axes nicely '''
    @ticker.FuncFormatter
    def date_formatter(x, pos):
        return (start_day + dt.timedelta(days=x)).strftime('%b-%d')
    ax.xaxis.set_major_formatter(date_formatter)
    ax.set_xlim([0, n_days])
    ax.yaxis.set_major_locator(plt.MaxNLocator(3))
    ax.get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    return
outputs = {'diagnoses':'New Diagnoses',
           'death': 'New Deaths',
           'infectious': 'Active Infections'}
letters = ['a','b','c']
for i, k in enumerate(outputs.keys()):
    l = letters[i]
    ax[l].plot(df_plot[k][0.500],color=colors[i])
    ax[l].fill_between(df_plot.index,
                        df_plot[k][0.250],
                        df_plot[k][0.750],
                        color=colors[i],
                        alpha=0.4,
                        label='50\% quantiles')
    ax[l].fill_between(df_plot.index,
                        df_plot[k][0.025],
                        df_plot[k][0.9750],
                        color=colors[i],
                        alpha=0.2,
                        label='95\% quantiles')
    if k != 'infectious':
        data_x = df_plot.index
        data_y = df[f"{k}_data"].values[0:len(data_x)]
        ax[l].plot(
        data_x,
        data_y,
        color='black',
        label='Data')
        ax[l].legend(edgecolor='black',loc='upper left')
        ax[l].set_title(outputs[k])
        format_ax(ax[l])

# handle infectious data
pop_size = 2.25e6
scan_file = '../../inputs/scanprev_5_21.csv'
scan = pd.read_csv(scan_file)
date_offset = pd.to_datetime([start_day for _ in range(len(scan))])
scan['since'] = pd.to_datetime(scan['to']) - date_offset
scan['to'] = pd.to_datetime(scan['to']) - date_offset

scan[['since','to']].mean(axis=1)
for i,r in scan.iterrows():
    label = "Data" if i==0 else None
    ts = np.mean([r['since'].days, r['to'].days])
    low  = r['lower']*pop_size
    high = r['upper']*pop_size
    mean = r['mean']*pop_size
    ax['c'].plot([ts]*2, [low, high], alpha=1.0, color='k', zorder=1000)
    ax['c'].plot(ts, mean, 'o', markersize=7, color='k', alpha=0.5, label=label, zorder=1000)
ax['c'].set_title('Active Infections')
format_ax(ax['c'])
ax['c'].legend(edgecolor='black',loc='upper left')


# plot posterior
posterior = pd.read_csv('../posterior-samples.csv')[list(bounds.keys())]
new_letters = ['d','e','f','g']
mapping = sc.objdict({
    'beta'  : r'Overall $\beta$',
    'bc_wc1': r'Work/community $\beta$ reduction',
    'bc_lf' : r'LTCF $\beta$ reduction',
    'tn'   : 'Symptomatic testing OR',
    })
keys = list(bounds.keys())
lb = [bounds[k][0] for k in keys]
ub = [bounds[k][1] for k in keys]

for i, key in enumerate(posterior.columns.tolist()):
    l = new_letters[i]
    posterior[key].hist(ax=ax[l],color='orange',bins=5,grid=False)
    ax[l].set_ylabel('Count')
    ax[l].set_xlabel(mapping[key])
    ax[l].set_xlim(bounds[key])


# label subplots
from matplotlib.transforms import ScaledTranslation
def add_fig_labels(position,label):
    ax[position].text(
        0.0, 1.0, label, transform=(
            ax[position].transAxes + ScaledTranslation(-20/72, +7/72, fig.dpi_scale_trans)),
        fontsize='large', va='bottom', fontfamily='serif')
    

add_fig_labels('a','a)')
add_fig_labels('b','b)')
add_fig_labels('c','c)')
add_fig_labels('d','d)')

fig.tight_layout()
fig.savefig('../figs/posterior-samples.pdf')

In [ ]:
# age hists
import matplotlib.pylab as pl
import covasim as cv
import numpy as np
agehists = []
full_sims = cv.load('../posterior/posterior.sims')
fig, ax = plt.subplots(ncols=2,figsize=(6,4))

for s,sim in enumerate(full_sims.sims):
    agehist = sim['analyzers'][0]
    if s == 0:
        age_data = agehist.data
    agehists.append(agehist.hists[-1])

# Observed data
x = age_data['age'].values
pos = age_data['cum_diagnoses'].values
death = age_data['cum_deaths'].values

# Model outputs
mposlist = []
mdeathlist = []
for hists in agehists:
    mposlist.append(hists['diagnosed'])
    mdeathlist.append(hists['dead'])
mposarr = np.array(mposlist)
mdeatharr = np.array(mdeathlist)

low_q = 0.025
high_q = 0.975
mpbest = pl.median(mposarr, axis=0)
mplow  = pl.quantile(mposarr, q=low_q, axis=0)
mphigh = pl.quantile(mposarr, q=high_q, axis=0)
mdbest = pl.median(mdeatharr, axis=0)
mdlow  = pl.quantile(mdeatharr, q=low_q, axis=0)
mdhigh = pl.quantile(mdeatharr, q=high_q, axis=0)

w = 4
off = 2

# Insets
x0s, y0s, dxs, dys = 0.25, 0.3, 0.2, 0.4

c1 = [0.3,0.3,0.6]
c2 = [0.6,0.7,0.9]
xx = x+w-off
ax[0].bar(x-off,pos, width=w, label='Data', facecolor=c1)
ax[0].bar(xx, mpbest, width=w, label='Model', facecolor=c2)
for i,ix in enumerate(xx):
    ax[0].plot([ix,ix], [mplow[i], mphigh[i]], c='k')
ax[0].set_xticks(np.arange(0,81,20))
ax[0].set_xlabel('Age')
ax[0].set_ylabel('Cases')
sc.boxoff(ax[0])
ax[0].legend(edgecolor='black',loc='upper left')


c1 = [0.5,0.0,0.0]
c2 = [0.9,0.4,0.3]
ax[1].bar(x-off,death, width=w, label='Data', facecolor=c1)
ax[1].bar(x+w-off, mdbest, width=w, label='Model', facecolor=c2)
for i,ix in enumerate(xx):
    ax[1].plot([ix,ix], [mdlow[i], mdhigh[i]], c='k')
ax[1].set_xticks(np.arange(0,81,20))
ax[1].set_xlabel('Age')
ax[1].set_ylabel('Deaths')
sc.boxoff(ax[1])
ax[1].legend(edgecolor='black',loc='upper left')
sc.boxoff(ax[1])


fig.savefig('../figs/Supp/Age-Hists.pdf')

In [ ]:
# MCMC Chains
import arviz as az
mcmc_titles = {'diagnoses':'Diagnoses','death':'Deaths'}
for output in ['diagnoses','death']:
    mcmc = sc.load(f'../posterior/calibrated-{output}-full.pkl')
    az.plot_trace(mcmc,compact=True,figsize=(9,6),
              legend=True,chain_prop='color')
    plt.suptitle(mcmc_titles[output])
    plt.tight_layout()
    plt.savefig(f'../figs/Supp/MCMC_{output}.pdf')


In [ ]:
# make posterior in json form
import json
keys = list(bounds.keys())
posterior_json = sims[keys + ['rand_seed']].drop_duplicates()
posterior_json['index'] = posterior_json['rand_seed'].values
posterior_json['mismatch'] = np.nan
records = posterior_json.to_dict(orient='records')

parkeys = keys + ['rand_seed']
js_list = []
for record in records:
    js_list.append({'pars':{key:record[key] for key in parkeys},
                    'index':record['index'],
                    'mismatch':record['mismatch']})
with open("../posterior/abc_pars.json", "w") as outfile:
    # Write the JSON data to the file
    json.dump(js_list, outfile)